In [1]:
# %pip install --upgrade py2neo
# %pip install --upgrade openai

In [2]:
from py2neo import Graph


In [3]:
from itertools import groupby

In [4]:
import json


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


def parse_json(json_string):
  json_dict = json.loads(json_string)
  return json_dict


def dict_to_pretty_json(dictionary):
  pretty_json = json.dumps(dictionary, indent=2)
  return pretty_json


def write_dict_to_json(dictionary, file_path):
  with open(file_path, 'w') as json_file:
    json.dump(dictionary, json_file, indent=2)


In [5]:
import configparser

def read_ini_file(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    ini_dict = {section: dict(config.items(section))
                for section in config.sections()}
    return ini_dict


In [6]:
from openai import OpenAI

In [7]:
import re


def remove_java_comments(java_source):
    # Regular expression to match Java comments (both single-line and multi-line)
    pattern = r"(//.*?$)|(/\*.*?\*/)"

    # Remove comments using the regular expression
    java_source_without_comments = re.sub(
        pattern, "", java_source, flags=re.MULTILINE | re.DOTALL)

    return java_source_without_comments


In [8]:
def sentence(s):
  '''
  Capitalize the first letter of a string `s` and ensures that the string 
  ends with a period (if it's not already a sentence-ending punctuation).
  '''
  t = s.strip()
  if t[-1] in '.?!…~–—':
    return f'{t[0].upper()}{t[1:]}'
  else:
    return f'{t[0].upper()}{t[1:]}.'
  
sentence(' hello world~  ')

'Hello world~'

## Parameters

In [9]:
# If True: do not call the API, just print the prompts
only_print_prompt = False

In [10]:
secrets = read_ini_file('secrets.ini')
project_name = secrets['project']['name']
project_name

'k9mail'

In [11]:
project_desc = 'an open source email client for Android focused on making it easy to chew through large volumes of email'

## Connect to neo4j

To access knowledge graph extracted using javapers which is then loaded to neo4j graph database.

In [12]:
graph = Graph(secrets['neo4j']['url'], auth=(secrets['neo4j']['username'], secrets['neo4j']['password']))

## Connect to openai

In [ ]:
args = dict()

if 'apikey' in secrets['openai']:
  args['api_key'] = secrets['openai']['apikey']
if 'apibase' in secrets['openai']:
  args['base_url'] = secrets['openai']['apibase']
if 'model' in secrets['openai']:
  model = secrets['openai']['model']
else:
  model = "gpt-3.5-turbo"

(args['base_url'], model)

In [14]:
client = OpenAI(**args)
client.base_url

URL('https://api.openai.com/v1/')

In [15]:
# test the LLM server---create a completion
completion = client.chat.completions.create(
  model=model,
  messages=[{"role":"user","content":"What is your name?"}],
  temperature=0
)
# print the completion
print(completion.choices[0].message.content)

I am a language model AI created by OpenAI, so I do not have a personal name. You can refer to me as OpenAI or Assistant. How can I assist you today?


In [16]:
# layers = [
# 	{
# 		"name":"Presentation Layer",
# 		"class_characteristics": [
# 			"contains fields that represent UI objects such as buttons and text fields",
# 			"defines a UI markup template with markers of where to put data from model",
# 		],
# 		"method_characteristics": [
# 			"sets the attribute of UI objects based on information from other objects",
# 			"arranges UI objects in a particular layout",
# 			"observes changes in a data model and updates UI objects accordingly",
# 			"reacts to UI events by creating or invoking data model objects",
# 			"determines how to update a UI view according to the result of data model process result",
# 			"reacts to UI events by deciding what kind of action to initiate and delegates to a command to carry out the action ",
# 			"recognizes domain data and invokes the appropriate UI rendering transformations",
# 			"processes domain data element by element and transforms it into visual representation for the UI",
# 			"assembles model information in a presentation-oriented logical structure for the UI",
# 			"renders a presentation-oriented structure in a UI view markup", 
# 		]
# 	},
# 	{
# 		"name":"Service Layer",
# 		"class_characteristics": [
# 			"establishes a set of avaliable domain operations",
# 			"provides a coarse-grained interface that delegates to/coordinates finer-grained interfaces",
# 		],
# 		"method_characteristics": [
# 			"responds to user input, manipulates model, and causes the view to update appropriately",
# 			"decides which domain logic to run upon an event",
# 			"decides the view with which to display the result of a domain operation",
# 			"encapsulates business logic",
# 			"controls transactions and coordinates responses in the implementation of its operations",
# 			"wraps several calls to a domain model within one method",
# 		]
# 	},
# 	{
# 		"name":"Domain Layer",
# 		"class_characteristics": [
# 			"organizes business logic by procedures where each procedure handles a single request from the presentation",
# 			"handles the business logic for all rows in a database table or view",
# 			"groups data model with behavior that acts on that data",
# 			"represents a tabular data",
# 			"represents a serializable data",
# 			"consists of fields and getters and setters",
# 			"has simple or primitive fields",
# 		],
# 		"method_characteristics": [
# 			"makes calls to the data source",
# 			"represents some meaningful business rules or logic",
# 			"assembles information from multiple domain objects",
# 		]
# 	},
# 	{
# 		"name":"Data Source Layer",
# 		"class_characteristics": [
# 			"represents a single data record",
# 			"acts as a gateway to a data source",
# 			"has a simple interface consisting find, update, insert, and delete methods",
# 			"maintains a list of objects affected by a  transaction and coordinates the writing out of changes",
# 			"maintains an identity map to see which objects are already loaded from the data source",
# 			"wraps a record in a data source, encapsulates the data source access, and adds domain logic on that data",
# 			"is responsible for saving and loading to the data source and also for any domain logic that acts on the data",
# 		],
# 		"method_characteristics": [
# 			"performs conversion from data source-type to in-memory representation",
# 			"contains operations for insert/update/delete to a data source",
# 			"finds information from a data source and returns its representation",
# 			"returns a tabular data",
# 			"bundles changes to a data source as a single transaction",
# 			"performs inconsistency checking before commiting changes to a data source",
# 			"maps an object from the data source to an equivalent model object",
# 		]
# 	},
# ]

In [17]:
# layers

In [18]:
# prompt_template = '''Consider the context of an Android Java project "{project_name}": {project_desc}.

# The {class_type} `{class_name}` contains the method `{method_name}`:

# ```java
# {method_src}
# ```

# Which of the following statements could reflect the above method? There can be 0, 1, or more answers. Hint: when you see "domain", think "Android email client" and when you see "data source", it means a database, external file, or equivalent.

# {method_characteristics}

# Think step by step. The last line of your response must only be an array containing statement numbers that are true, e.g., `[1,3]`.
# '''

In [19]:
prompt1_template = '''Here are the layers in a layered software architecture and their responsibilities:

1. **Presentation Layer**: Manages the user interface, including defining UI elements and their behavior, displays information, reacting to user input, and updating UI views accordingly.
  
2. **Service Layer**: Orchestrates domain operations, encapsulates business logic, selects the appropriate business logic for a user request, and coordinates responses between the presentation and domain layers.

3. **Domain Layer**: Organizes and implements business logic, represents domain data and its behavior, and carries out the necessary computation for responding to user requests.

4. **Data Source Layer**: Communicates with databases, messaging systems, or other sources of data, performs CRUD operations, handles data conversion, and ensures data integrity before committing changes to the data source.

Consider the following Java method from {project_desc}:

```java
{method_src}
```

Reason about whether this method fits with each of the layers above. Think step by step. First, summarize what is the responsibility of the method. Then compare it to the layers above.'''

prompt2 = "In conclusion, state a single layer that you think fits this method the most. Just answer with the name of the layer and nothing else."

In [20]:
packages = { record['p']['qualifiedName'] for record in graph.run('match (p:Container)-[:contains]->(:Structure) where p.kind="package" return p') }

In [21]:
len(packages), packages

(38,
 {'com.fsck.k9',
  'com.fsck.k9.account',
  'com.fsck.k9.activity',
  'com.fsck.k9.activity.compose',
  'com.fsck.k9.activity.loader',
  'com.fsck.k9.activity.misc',
  'com.fsck.k9.activity.setup',
  'com.fsck.k9.autocrypt',
  'com.fsck.k9.cache',
  'com.fsck.k9.controller',
  'com.fsck.k9.crypto',
  'com.fsck.k9.fragment',
  'com.fsck.k9.helper',
  'com.fsck.k9.helper.jsoup',
  'com.fsck.k9.mailstore',
  'com.fsck.k9.mailstore.migrations',
  'com.fsck.k9.mailstore.util',
  'com.fsck.k9.message',
  'com.fsck.k9.message.extractors',
  'com.fsck.k9.message.html',
  'com.fsck.k9.message.quote',
  'com.fsck.k9.message.signature',
  'com.fsck.k9.notification',
  'com.fsck.k9.power',
  'com.fsck.k9.preferences',
  'com.fsck.k9.provider',
  'com.fsck.k9.remotecontrol',
  'com.fsck.k9.search',
  'com.fsck.k9.service',
  'com.fsck.k9.setup',
  'com.fsck.k9.ui',
  'com.fsck.k9.ui.compose',
  'com.fsck.k9.ui.crypto',
  'com.fsck.k9.ui.dialog',
  'com.fsck.k9.ui.message',
  'com.fsck.k9.ui.me

In [22]:
take_samples = False

In [23]:
# import random

# if take_samples:
#   num_methods = 0

#   samples = dict()

#   for pkg_name in sorted(list(packages)):

#     classes = [ record['c'] for record in graph.run('MATCH (p:Container)-[:contains]->(c:Structure) '
#                                                                     f'WHERE p.qualifiedName="{pkg_name}" AND p.kind="package" '
#                                                                     'RETURN c') ]
#     top_classes = [c for c in classes if not '$' in c['qualifiedName']]
#     class_samples = random.sample(top_classes, min(len(top_classes),3))
    
#     samples[pkg_name] = dict()
#     for clss in class_samples:

#       class_name = clss['qualifiedName']
#       kind = clss['kind']
#       methods = [ record['m'] for record in graph.run('MATCH (c:Structure)-[:hasScript]->(m:Operation) '
#                                                       f'WHERE c.qualifiedName="{class_name}" AND m.visibility="public" AND m.kind="method" '
#                                                       'return m') ]
#       ok_methods = [m for m in methods if 300<len(m['sourceText'])<3000]
#       method_samples = random.sample(ok_methods, min(len(ok_methods),10))
#       # print(len(method_samples), [(clss['qualifiedName'], m['simpleName']) for m in method_samples])
#       num_methods += len(method_samples)

#       samples[pkg_name][class_name] = method_samples
#   print(samples)


In [24]:
# if take_samples:
# 	write_dict_to_json(samples, 'k9mail-samples.json')

In [25]:
# if not take_samples:
# 	samples = read_json_file('k9mail-samples.json')
# samples

In [26]:
num_methods = 0

samples = dict()

for pkg_name in sorted(list(packages)):
    classes = [ record['c'] for record in graph.run('MATCH (p:Container)-[:contains]->(c:Structure) '
                                                                    f'WHERE p.qualifiedName="{pkg_name}" AND p.kind="package" '
                                                                    'RETURN c') ]
    # top_classes = [c for c in classes if not '$' in c['qualifiedName']]
    # class_samples = random.sample(top_classes, min(len(top_classes),3))
    
    samples[pkg_name] = dict()
    for clss in classes:

      class_name = clss['qualifiedName']
      kind = clss['kind']
      methods = [ record['m'] for record in graph.run('MATCH (c:Structure)-[:hasScript]->(m:Operation) '
                                                      f'WHERE c.qualifiedName="{class_name}" AND m.visibility="public" AND m.kind="method" '
                                                      'return m') ]
    #   ok_methods = [m for m in methods if 300<len(m['sourceText'])<3000]
    #   method_samples = random.sample(ok_methods, min(len(ok_methods),10))
      # print(len(method_samples), [(clss['qualifiedName'], m['simpleName']) for m in method_samples])
      num_methods += len(methods)

      samples[pkg_name][class_name] = methods
# print(samples)
num_methods

2693

In [27]:
samples.keys()

dict_keys(['com.fsck.k9', 'com.fsck.k9.account', 'com.fsck.k9.activity', 'com.fsck.k9.activity.compose', 'com.fsck.k9.activity.loader', 'com.fsck.k9.activity.misc', 'com.fsck.k9.activity.setup', 'com.fsck.k9.autocrypt', 'com.fsck.k9.cache', 'com.fsck.k9.controller', 'com.fsck.k9.crypto', 'com.fsck.k9.fragment', 'com.fsck.k9.helper', 'com.fsck.k9.helper.jsoup', 'com.fsck.k9.mailstore', 'com.fsck.k9.mailstore.migrations', 'com.fsck.k9.mailstore.util', 'com.fsck.k9.message', 'com.fsck.k9.message.extractors', 'com.fsck.k9.message.html', 'com.fsck.k9.message.quote', 'com.fsck.k9.message.signature', 'com.fsck.k9.notification', 'com.fsck.k9.power', 'com.fsck.k9.preferences', 'com.fsck.k9.provider', 'com.fsck.k9.remotecontrol', 'com.fsck.k9.search', 'com.fsck.k9.service', 'com.fsck.k9.setup', 'com.fsck.k9.ui', 'com.fsck.k9.ui.compose', 'com.fsck.k9.ui.crypto', 'com.fsck.k9.ui.dialog', 'com.fsck.k9.ui.message', 'com.fsck.k9.ui.messageview', 'com.fsck.k9.view', 'com.fsck.k9.widget.list'])

In [28]:
samples['com.fsck.k9.ui']

{'com.fsck.k9.ui.EolConvertingEditText': [Node('Operation', kind='method', metaSrc='source code', name='setCharacters(java.lang.CharSequence)', qualifiedName='com.fsck.k9.ui.EolConvertingEditText.setCharacters(java.lang.CharSequence)', simpleName='setCharacters(java.lang.CharSequence)', sourceText='/**\r\n * Sets the string value of the EolConvertingEditText. Any line endings\r\n * in the string will be converted to {@code \\n}.\r\n *\r\n * @param text\r\n */\r\npublic void setCharacters(java.lang.CharSequence text) {\r\n    setText(text.toString().replace("\\r\\n", "\\n"));\r\n}', visibility='public'),
  Node('Operation', kind='method', metaSrc='source code', name='getCharacters()', qualifiedName='com.fsck.k9.ui.EolConvertingEditText.getCharacters()', simpleName='getCharacters()', sourceText='/**\r\n * Return the text the EolConvertingEditText is displaying.\r\n *\r\n * @return A string with any line endings converted to {@code \\r\\n}.\r\n */\r\npublic java.lang.String getCharacters(

In [29]:
only_print_prompt = False

In [30]:
import time

timestr = time.strftime("%Y%m%d-%H%M%S")
timestr

'20240411-161946'

In [31]:
with open(f'layerinator-{timestr}.log', 'a') as file:

  results = dict()
  kind = 'class'
  for pkg_name in samples.keys():
    results[pkg_name] = dict()
    for class_name in samples[pkg_name].keys():
      results[pkg_name][class_name] = dict()
      for method in samples[pkg_name][class_name]:
        method_name = method['simpleName']
        results[pkg_name][class_name][method_name] = dict()
        file.write(f"# {pkg_name}, {class_name}, {method_name}")
        file.write("\n\n")
        # file.write(method['sourceText'])
        # file.write("\n")
        # for layer in layers:
        #   print(f'## {layer["name"]}')
        #   print()
        #   prompt = prompt_template.format(
        #       # layer_name=layer["name"],
        #       # layer_responsibility=layer["responsibility"],
        #       method_characteristics="\n".join(
        #           [f"  {i+1}. {layer['method_characteristics'][i]}" for i in range(len(layer["method_characteristics"]))]),
        #       project_name=project_name,
        #       project_desc=project_desc,
        #       class_type=kind,
        #       class_name=class_name,
        #       method_name=method_name,
        #       # method_desc = goals[pkg_name]['classes'][row["c.qualifiedName"]]['methods'][method_name]['description'],
        #       method_src=remove_java_comments(method["sourceText"]))
        prompt1 = prompt1_template.format(
          project_desc=project_desc,
          method_src=method["sourceText"]
        )
        if only_print_prompt:
          file.write(prompt1)
          file.write("\n\n")
        else:
          response = None
          try:
            response = client.chat.completions.create(
                model=model,
                messages=[{
                  "role": "user",
                  "content": prompt1
                }],
                temperature=0)
            ast_message = response.choices[0].message
            
            file.write("[USER]\n\n")
            file.write(prompt1)
            file.write("\n\n")
            file.write("[LLM]\n\n")
            file.write(ast_message.content)
            file.write("\n\n")

            response = client.chat.completions.create(
                model=model,
                messages=[{
                  "role": "user",
                  "content": prompt1
                }, 
                ast_message,
                {
                  "role": "user",
                  "content": prompt2
                }],
                temperature=0)
            answer = response.choices[0].message.content
            
            file.write("[USER]\n\n")
            file.write(prompt2)
            file.write("\n\n")
            file.write("[LLM]\n\n")
            file.write(answer)
            file.write("\n\n")

            # print(answer)
            # results[pkg_name][class_name][method_name][layer['name']] = parse_json(answer.split('\n')[-1])
            results[pkg_name][class_name][method_name]['layer'] = answer
          except:
            answer = None
            file.write(str(response) if response else "no response")
            file.write("\n\n")
            # results[pkg_name][class_name][method_name][layer['name']] = []
            results[pkg_name][class_name][method_name]['layer'] = "undefined"
          # print(answer)
          # print()
        file.write("===============================================\n\n")
        # print(dict_to_pretty_json(results[pkg_name][class_name][method_name]))
        # print()
        # break
      file.write("CLASS RESULT:\n\n")
      file.write(dict_to_pretty_json(results[pkg_name][class_name]))
      file.write("\n\n")
      file.flush()
      # break
    file.write("PACKAGE RESULT:\n\n")
    file.write(dict_to_pretty_json(results[pkg_name]))
    file.write("\n\n")
    # break
  file.write("ALL RESULTS:\n\n")
  file.write(dict_to_pretty_json(results))


In [32]:
if not only_print_prompt:
  write_dict_to_json(results, f"layerinator-v2/{project_name}-layers-{timestr}.json")

In [33]:
def count_layer_occurrences(input_dict):
    layer_count = {}

    for method, details in input_dict.items():
        layer = details.get("layer")
        if layer:
            if layer in layer_count:
                layer_count[layer] += 1
            else:
                layer_count[layer] = 1

    return layer_count

In [34]:
results

{'com.fsck.k9': {'com.fsck.k9.Throttle$MyTimerTask': {'run()': {'layer': 'Service Layer'},
   'cancel()': {'layer': 'Presentation Layer'}},
  'com.fsck.k9.PRNGFixes$LinuxPRNGSecureRandomProvider': {},
  'com.fsck.k9.Throttle': {'cancelScheduledCallback()': {'layer': 'Execution Layer'},
   'onEvent()': {'layer': 'Service Layer'}},
  'com.fsck.k9.K9$SplitViewMode': {},
  'com.fsck.k9.PRNGFixes': {'apply()': {'layer': 'Utility Layer'}},
  'com.fsck.k9.K9$Intents$EmailReceived': {},
  'com.fsck.k9.BaseAccount': {'setEmail(java.lang.String)': {'layer': 'Domain Layer'},
   'getEmail()': {'layer': 'Domain Layer'},
   'getUuid()': {'layer': 'Domain Layer'},
   'getDescription()': {'layer': 'Domain Layer'},
   'setDescription(java.lang.String)': {'layer': 'Domain Layer'}},
  'com.fsck.k9.AccountStats': {},
  'com.fsck.k9.K9$Intents$Share': {},
  'com.fsck.k9.Account$ShowPictures': {},
  'com.fsck.k9.Throttle$MyTimerTask$HandlerRunnable': {'run()': {'layer': 'Service Layer'}},
  'com.fsck.k9.K9$

In [35]:
for package in results:
	for clss in results[package]:
		results[package][clss]['layers'] = count_layer_occurrences(results[package][clss])

results

{'com.fsck.k9': {'com.fsck.k9.Throttle$MyTimerTask': {'run()': {'layer': 'Service Layer'},
   'cancel()': {'layer': 'Presentation Layer'},
   'layers': {'Service Layer': 1, 'Presentation Layer': 1}},
  'com.fsck.k9.PRNGFixes$LinuxPRNGSecureRandomProvider': {'layers': {}},
  'com.fsck.k9.Throttle': {'cancelScheduledCallback()': {'layer': 'Execution Layer'},
   'onEvent()': {'layer': 'Service Layer'},
   'layers': {'Execution Layer': 1, 'Service Layer': 1}},
  'com.fsck.k9.K9$SplitViewMode': {'layers': {}},
  'com.fsck.k9.PRNGFixes': {'apply()': {'layer': 'Utility Layer'},
   'layers': {'Utility Layer': 1}},
  'com.fsck.k9.K9$Intents$EmailReceived': {'layers': {}},
  'com.fsck.k9.BaseAccount': {'setEmail(java.lang.String)': {'layer': 'Domain Layer'},
   'getEmail()': {'layer': 'Domain Layer'},
   'getUuid()': {'layer': 'Domain Layer'},
   'getDescription()': {'layer': 'Domain Layer'},
   'setDescription(java.lang.String)': {'layer': 'Domain Layer'},
   'layers': {'Domain Layer': 5}},
  '

In [36]:
def sum_layer_counts(input_dicts):
    layer_count = {}

    for class_name, details in input_dicts.items():
        layers = details.get("layers", {})
        for layer, count in layers.items():
            if layer in layer_count:
                layer_count[layer] += count
            else:
                layer_count[layer] = count

    return layer_count

In [37]:
for package in results:
	results[package]['layers'] = sum_layer_counts(results[package])

results

{'com.fsck.k9': {'com.fsck.k9.Throttle$MyTimerTask': {'run()': {'layer': 'Service Layer'},
   'cancel()': {'layer': 'Presentation Layer'},
   'layers': {'Service Layer': 1, 'Presentation Layer': 1}},
  'com.fsck.k9.PRNGFixes$LinuxPRNGSecureRandomProvider': {'layers': {}},
  'com.fsck.k9.Throttle': {'cancelScheduledCallback()': {'layer': 'Execution Layer'},
   'onEvent()': {'layer': 'Service Layer'},
   'layers': {'Execution Layer': 1, 'Service Layer': 1}},
  'com.fsck.k9.K9$SplitViewMode': {'layers': {}},
  'com.fsck.k9.PRNGFixes': {'apply()': {'layer': 'Utility Layer'},
   'layers': {'Utility Layer': 1}},
  'com.fsck.k9.K9$Intents$EmailReceived': {'layers': {}},
  'com.fsck.k9.BaseAccount': {'setEmail(java.lang.String)': {'layer': 'Domain Layer'},
   'getEmail()': {'layer': 'Domain Layer'},
   'getUuid()': {'layer': 'Domain Layer'},
   'getDescription()': {'layer': 'Domain Layer'},
   'setDescription(java.lang.String)': {'layer': 'Domain Layer'},
   'layers': {'Domain Layer': 5}},
  '

In [46]:
write_dict_to_json(results, f"layerinator-v2/{project_name}-layers-recap-{timestr}.json")

In [38]:
rows = []

for package in results:
	for clss in [c for c in results[package] if c != 'layers']:
		for method in [m for m in results[package][clss] if m != 'layers']:
			rows.append((package,clss,method,results[package][clss][method]['layer']))

rows

[('com.fsck.k9', 'com.fsck.k9.Throttle$MyTimerTask', 'run()', 'Service Layer'),
 ('com.fsck.k9',
  'com.fsck.k9.Throttle$MyTimerTask',
  'cancel()',
  'Presentation Layer'),
 ('com.fsck.k9',
  'com.fsck.k9.Throttle',
  'cancelScheduledCallback()',
  'Execution Layer'),
 ('com.fsck.k9', 'com.fsck.k9.Throttle', 'onEvent()', 'Service Layer'),
 ('com.fsck.k9', 'com.fsck.k9.PRNGFixes', 'apply()', 'Utility Layer'),
 ('com.fsck.k9',
  'com.fsck.k9.BaseAccount',
  'setEmail(java.lang.String)',
  'Domain Layer'),
 ('com.fsck.k9', 'com.fsck.k9.BaseAccount', 'getEmail()', 'Domain Layer'),
 ('com.fsck.k9', 'com.fsck.k9.BaseAccount', 'getUuid()', 'Domain Layer'),
 ('com.fsck.k9',
  'com.fsck.k9.BaseAccount',
  'getDescription()',
  'Domain Layer'),
 ('com.fsck.k9',
  'com.fsck.k9.BaseAccount',
  'setDescription(java.lang.String)',
  'Domain Layer'),
 ('com.fsck.k9',
  'com.fsck.k9.Throttle$MyTimerTask$HandlerRunnable',
  'run()',
  'Service Layer'),
 ('com.fsck.k9',
  'com.fsck.k9.K9',
  'messageLi

In [39]:
header = ("package", "class", "method", "layer")

In [40]:
import csv

with open(f"layerinator-v2/{project_name}-layers1-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [41]:
rows = []

for package in results:
	for clss in [c for c in results[package] if c != 'layers']:
		for layer in results[package][clss]['layers']:
			rows.append((package,clss,layer,results[package][clss]['layers'][layer]))

rows

[('com.fsck.k9', 'com.fsck.k9.Throttle$MyTimerTask', 'Service Layer', 1),
 ('com.fsck.k9', 'com.fsck.k9.Throttle$MyTimerTask', 'Presentation Layer', 1),
 ('com.fsck.k9', 'com.fsck.k9.Throttle', 'Execution Layer', 1),
 ('com.fsck.k9', 'com.fsck.k9.Throttle', 'Service Layer', 1),
 ('com.fsck.k9', 'com.fsck.k9.PRNGFixes', 'Utility Layer', 1),
 ('com.fsck.k9', 'com.fsck.k9.BaseAccount', 'Domain Layer', 5),
 ('com.fsck.k9',
  'com.fsck.k9.Throttle$MyTimerTask$HandlerRunnable',
  'Service Layer',
  1),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Presentation Layer', 40),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Service Layer', 45),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Domain Layer', 35),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Data Source Layer', 10),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Utility Layer', 5),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'None', 2),
 ('com.fsck.k9', 'com.fsck.k9.EmailAddressValidator', 'Domain Layer', 3),
 ('com.fsck.k9', 'com.fsck.k9.Account$SortType', 'Domain Layer', 1),
 ('com.fsck.k9',

In [42]:
header = ("package", "class", "layer", "count")

In [43]:
import csv

with open(f"layerinator-v2/{project_name}-layers2-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)

In [44]:
rows = []

for package in results:
	for clss in [c for c in results[package] if c != 'layers']:
		for layer in results[package][clss]['layers']:
			rows.append((package,clss,layer,results[package][clss]['layers'][layer]/sum(results[package][clss]['layers'].values())))

rows

[('com.fsck.k9', 'com.fsck.k9.Throttle$MyTimerTask', 'Service Layer', 0.5),
 ('com.fsck.k9',
  'com.fsck.k9.Throttle$MyTimerTask',
  'Presentation Layer',
  0.5),
 ('com.fsck.k9', 'com.fsck.k9.Throttle', 'Execution Layer', 0.5),
 ('com.fsck.k9', 'com.fsck.k9.Throttle', 'Service Layer', 0.5),
 ('com.fsck.k9', 'com.fsck.k9.PRNGFixes', 'Utility Layer', 1.0),
 ('com.fsck.k9', 'com.fsck.k9.BaseAccount', 'Domain Layer', 1.0),
 ('com.fsck.k9',
  'com.fsck.k9.Throttle$MyTimerTask$HandlerRunnable',
  'Service Layer',
  1.0),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Presentation Layer', 0.291970802919708),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Service Layer', 0.3284671532846715),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Domain Layer', 0.25547445255474455),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Data Source Layer', 0.072992700729927),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'Utility Layer', 0.0364963503649635),
 ('com.fsck.k9', 'com.fsck.k9.K9', 'None', 0.014598540145985401),
 ('com.fsck.k9', 'com.fsck.k9.EmailAddre

In [45]:
import csv

with open(f"layerinator-v2/{project_name}-layers3-{timestr}.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(rows)